Note:
    
    1. When listing info about folders and files the level should be INFO.
    
    2. When the passing value for folder or file is not valid (folder or file doesn’t exist) the level should be WARNING.
    
    3. Any computation errors should be captured (e.g. total storage of files, folders, drives cannot be negative) and classified under the ERROR level.
    
    4. When Python can NOT read the OS structure, the level should be CRITICAL.
    
    5. Any other activity/message of the logger should have a level of DEBUG.

In [1]:
# NOTE: - To Handle Initial Logging And Args Logic
# REVIEW: - Check if Windows Systems are 100% compatible

import logging
from constants import D, DRV, L, FLD, F, FIL, T, TYP, LOG_FORMAT, LOG_DATE_FORMAT

# NOTE: - Logger Initilized
logging.basicConfig(filename='dump.log', level=logging.DEBUG, format=LOG_FORMAT, datefmt=LOG_DATE_FORMAT)
logging.debug("Logging Initilized")

import argparse
from info_logger import log_drives, log_drive, log_folders, log_folder, log_files, log_file, log_file_types

# SECTION ARGS Initilization
# NOTE: - Set up Argparse paramaeters

parser = argparse.ArgumentParser()
parser.add_argument(D, help='list all the drives of the machine with additional info')
parser.add_argument(DRV, help='list additional info of single drive')

parser.add_argument(L, help='list all the folders of the machine with additional info:')
parser.add_argument(FLD, help='list additional info of single folder')

parser.add_argument(F, help='list all the files of the machine with additional info')
parser.add_argument(FIL, help='list additional info of single file')

parser.add_argument(T, help='list all the types of files of the machine with additional info')
parser.add_argument(TYP, help='list additional info of type of single file')
logging.debug("Args Added")

# In-Code arg passing (For Testing)
args = parser.parse_args(args=['-t', 'T'])

# SECTION ARGS Handeling
# NOTE: - Beginning Args Parameters Handeling
logging.debug("Beginning Args Handeling...")

List Drives

    ‘-d’: To list all the drives of the machine with the following info:
        - Drive name / letter
        - Total number of directories
        - Total number of files
        - Total allocated, used, free storage
    A positional argument ‘drv’ that will indicate the name of a single drive to report the above info.

In [2]:
from drive_dump import dump_drives, dump_drive

# SECTION -d and --drv
# NOTE: - -d args
if args.d:
    logging.debug('-d argument was passed')
    logging.debug("Dumping Local Computer Drives info")

    drives = dump_drives()
    if drives is None:
        logging.critical('Program was unable to process request')
        logging.error("Could Not Dump Drive Info")
    else: 
        log_drives(drives)
       

# NOTE: - -drv args
if args.drv:
    logging.debug('-drv argument was passed')
    logging.debug("Dumping info of %s Drive" % args.drv)
    drive = dump_drive(args.drv)
    if drive is None:
        logging.critical('Program was unable to process %s path' % args.drv)
    else:
        log_drive(drive)

Lists Folders

    l’: To list all the folders in a given drive with the following info:
        - Folder name
        - Total number of files per folder
        - Total storage used per folder and sum of all storage for all folders.
    A positional argument ‘fld’ that will pass the folder name to report the above info.

In [3]:
from folder_dump import dump_folder, dump_folders

# SECTION -l and --fld
# NOTE: - -l args
if args.l:
    logging.debug('-l argument was passed')
    logging.debug('Dumping the info of %s drive' % args.l )

    folders = dump_folders(args.l)
    if folders is None:
        logging.critical("The program was unable to obtain folder info ")
        logging.error("Could Not Dump folder info")
    else:
        log_folders(folders)

# NOTE: - --fld args
if args.fld:
    logging.debug('--fld argument was passed')
    logging.debug('Dumping the info of %s folder' % args.fld)

    folder = dump_folder(args.fld)
    if folder is None:
        logging.critical('The program was unable to obtain info from %s' % args.fld)  
    else:
         log_folder(folder)

Lists Files

    ‘-f’: To list all the files of the machine with the following info:
        - File name
        - File type (‘py’, ‘ipynb’, ‘exe’, ‘txt’, ‘csv’, ‘pdf’, ‘other’)
        - File size
        - Date/Time stamp of the file
    A positional argument ‘fil’ that will pass the file name to report the above info.

In [4]:
from file_dump import dump_files, dump_file

# SECTION -f and --fil
# NOTE: - -f args
if args.f:
    logging.debug('-f argument was passed')
    logging.debug("Dumping the info of this machine's files...Hold your butcheeks")
    files = dump_files()
    if files is None:
        logging.critical('The program was unable to obtain files info from this machine')
        logging.error("Could not dump this machine's file's info") 
    else:
        log_files(files)

# NOTE: - --fil args
if args.fil:
    logging.debug('--fil argument was passed')
    logging.debug('Dumping the info of %s' % args.fil)
    file = dump_file(args.fil)
    if file is None:
        logging.critical('The program was unable to obtain info from %s' % args.fil)
    else:
        log_file(file)

Lists File Types

    ‘-t’: To list all the types of files exist in the machine with the following info:
        - File type
        - Total number of files per file type
        - Total storage used per file type
    A positional argument ‘typ’ that will indicate the type of file to report the above info.

In [5]:
from file_type_dump import dump_file_types, dump_file_type

# SECTION -t and --typ
# NOTE: - -t args
if args.t:
    logging.debug('-t argument was passed')
    logging.debug("list all type and the total storage")
    file_types = dump_file_types()
    if file_types is None:
        logging.critical('The program was unable to obtain file types info from this machine')
        logging.error("Could not dump this machine's file types info") 
    else:
        log_file_types(file_types)

# NOTE: - -typ args
if args.typ:
    logging.debug('--typ argument was passed')
    logging.debug("list all type and the total storage")
    file_types = dump_file_type(args.typ)
    if file_types is None:
        logging.critical('The program was unable to obtain info from %s' % args.fil) 
    else:
        log_file_types(file_types)